

### Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Assignment)

This notebook is for you to practice skills during lecture.

Today's guided module project and assignment will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills.

## Sections
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy
* <a href="#p4">Part 4</a>: Post Lecture Assignment

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### Load Competition Data

In [1]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

# You may need to change the path
train = pd.read_csv('/home/jack/Desktop/Kaggle-NLP/train.csv')
test = pd.read_csv('/home/jack/Desktop/Kaggle-NLP/test.csv')

In [2]:
train.head()

,id,description,ratingCategory
0,1321,"\nSometimes, when whisky is batched, a few lef...",1
1,3861,\nAn uncommon exclusive bottling of a 6 year o...,0
2,655,\nThis release is a port version of Amrut’s In...,1
3,555,\nThis 41 year old single cask was aged in a s...,1
4,1965,"\nQuite herbal on the nose, with aromas of dri...",1


<h2>Cleaning</h2>

In [3]:
# get rid of \n in description
train['description'] = [x.replace('\n','') for x in train['description']]
test['description'] = [x.replace('\n','') for x in test['description']]

In [4]:
target = 'ratingCategory'
features = 'description'
X_train = train[features]
y_train = train[target]
X_test = test[features]
#y_test = test[target]

### Define Pipeline Components

In [5]:
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
clf = RandomForestClassifier()

pipe = Pipeline([('vect', vect), ('clf', clf)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [6]:
parameters = {
    'vect__max_df': (.75, 1.0),
    'vect__max_features': (1000, 2000, None),
    'clf__max_depth':(5,10,15,20),
    'clf__n_estimators': (50, 100, 200)
}

grid_search = GridSearchCV(pipe, parameters, cv=5, n_jobs=-3, verbose=1, random_state=42)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=-3)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-3)]: Done  42 tasks      | elapsed:   15.2s
[Parallel(n_jobs=-3)]: Done 192 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-3)]: Done 360 out of 360 | elapsed:  2.9min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 2),
                                                        no

In [7]:
grid_search.best_score_

0.7301224289472188

In [8]:
grid_search.best_params_

{'clf__max_depth': 20,
 'clf__n_estimators': 50,
 'vect__max_df': 0.75,
 'vect__max_features': 1000}

### Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so you only submit if you feel you cannot achieve higher test accuracy. For this competition the max daily submissions are capped at **20**. Submit for each demo and for your assignment. 

In [9]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [10]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [11]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [12]:
subNumber = 0

In [13]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'/home/jack/Desktop/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

You're trying to achieve a minimum of 70% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


### Define Pipeline Components

In [14]:
svd = TruncatedSVD(n_components=100,
                   algorithm='randomized',
                   n_iter=10)
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
lsi = Pipeline([('vect', vect), ('svd', svd)])

clf = RandomForestClassifier()
pipe = Pipeline([('lsi', lsi), ('clf', clf)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [16]:
parameters = {
    'lsi__svd__n_components': [10,100,250],
    'lsi__vect__max_df': (0.75, 1.0),
    'lsi__vect__max_features': (1000, 2000, None),
    'clf__max_depth':(5,10,15,20)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=1, random_state=42)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  38 tasks      | elapsed:   24.4s
[Parallel(n_jobs=-1)]: Done 188 tasks      | elapsed:  7.0min
[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed: 14.4min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('lsi',
                                        Pipeline(memory=None,
                                                 steps=[('vect',
                                                         TfidfVectorizer(analyzer='word',
                                                                         binary=False,
                                                                         decode_error='strict',
                                                                         dtype=<class 'numpy.float64'>,
                                                                         encoding='utf-8',
                                                                         input='content',
                                                                         lowercase=True,
                                                                         max_df=1.0,
             

In [17]:
print("Best score:", grid_search.best_score_)
print(grid_search.best_params_)

Best score: 0.723029570286665
{'clf__max_depth': 10, 'lsi__svd__n_components': 10, 'lsi__vect__max_df': 1.0, 'lsi__vect__max_features': 2000}


### Make a Submission File

In [18]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [19]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [20]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [21]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model
subNumber += 1
submission.to_csv(f'/home/jack/Desktop/submission{subNumber}.csv', index=False)

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

## Follow Along

In [22]:
# Continue Word Embedding Work Here
import spacy
nlp = spacy.load("en_core_web_lg")

In [23]:
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

X = get_word_vectors(train['description'])

In [24]:
# Apply to your Dataset

from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier

from scipy.stats import randint

gbc = GradientBoostingClassifier()

param_dist = {
    'max_depth' : [3, 5, 7, 10],
    'min_samples_leaf': [2, 5, 8, 12]
}

grid_search = GridSearchCV(gbc, param_dist, cv=5, n_jobs=-3, verbose=1, random_state=42)
grid_search.fit(X, train['ratingCategory'])

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-3)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-3)]: Done  42 tasks      | elapsed: 31.2min
[Parallel(n_jobs=-3)]: Done  80 out of  80 | elapsed: 76.6min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=GradientBoostingClassifier(ccp_alpha=0.0,
                                                  criterion='friedman_mse',
                                                  init=None, learning_rate=0.1,
                                                  loss='deviance', max_depth=3,
                                                  max_features=None,
                                                  max_leaf_nodes=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=2,
                                                  min_weight_fraction_leaf=0.0,
                                                  n_estimators=100,
                                                  n_iter_no_change=None,
         

In [25]:
print("Best score:", grid_search.best_score_)
print(grid_search.best_params_)

Best score: 0.7457781315744584
{'max_depth': 7, 'min_samples_leaf': 12}


### Make a Submission File

In [26]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [27]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [28]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model
subNumber += 1
submission.to_csv(f'/home/jack/Desktop/submission{subNumber}.csv', index=False)


In [29]:
# Submission accuracy best: 77.12%
# Reason: Forgot to set a random state
#         and was blessed by the Kaggle gods

# Consistent accuracy best: 75%

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Post Lecture Assignment
<a id="p4"></a>

Your primary assignment this afternoon is to achieve a minimum of 70% accuracy on the Kaggle competition. Once you have achieved 70% accuracy, please work on the following: 

1. Research "Sentiment Analysis". Provide answers in markdown to the following questions: 
    - What is "Sentiment Analysis"? 
    - Is Document Classification different than "Sentiment Analysis"? Provide evidence for your response
    - How do create labeled sentiment data? Are those labels really sentiment?
    - What are common applications of sentiment analysis?
2. Research our why word embeddings worked better for the lecture notebook than on the whiskey competition.
    - This [text classification documentation](https://developers.google.com/machine-learning/guides/text-classification/step-2-5) from Google might be of interest
    - Neural Networks are becoming more popular for document classification. Why is that the case?

<h1>Questions</h2>

<h2>What is sentiment analysis?</h2>
<p>Sentiment Analysis is the most common text classification tool that analyses an incoming message and tells whether the underlying sentiment is positive, negative our neutral. A sentiment analysis system for text analysis combines natural language processing (NLP) and machine learning techniques to assign weighted sentiment scores to the entities, topics, themes and categories within a sentence or phrase.</p>

<h2>What is document classification?</h2>
<p>Document classification is an example of Machine Learning in the form of Natural Language Processing (NLP). By classifying text, we are aiming to assign one or more classes or categories to a document, making it easier to manage and sort.</p>

<h2>Is Document Classification different than Sentiment Analysis?</h2>
<p>Both of these techniques are used to classify text but how they go about it is slightly different. Document classification compares a document to other documents to see which is most like it, while sentiment analysis compares the document against the structure of human language to make classifications.</p>

<h2>How do create labeled sentiment data? Are those labels really sentiment?</h2>
<p></p>

<h2>What are common applications of sentiment analysis?</h2>
<p>Brand monitoring and reputation management is the most common use of sentiment analysis across different markets.</p>

<h2>Why did word embeddings work better for the lecture notebook than on the whiskey competition?</h2>